In [8]:
import os
import codecs
import collections
import numpy as np
from six.moves import cPickle

# This is for loading TEXT!
class TextLoader():
    def __init__(self, data_dir, batch_size, seq_length):
        self.data_dir = data_dir
        self.batch_size = batch_size
        self.seq_length = seq_length

        input_file = os.path.join(data_dir, "input.txt")
        vocab_file = os.path.join(data_dir, "vocab.pkl")
        tensor_file = os.path.join(data_dir, "data.npy")

        if not (os.path.exists(vocab_file) and os.path.exists(tensor_file)):
            print("reading text file")
            self.preprocess(input_file, vocab_file, tensor_file)
        else:
            print("loading preprocessed files")
            self.load_preprocessed(vocab_file, tensor_file)
        self.create_batches()
        self.reset_batch_pointer()

    def preprocess(self, input_file, vocab_file, tensor_file):
        with codecs.open(input_file, "r", encoding="utf8") as f:
            data = f.read()
        counter = collections.Counter(data)
        count_pairs = sorted(counter.items(), key=lambda x: -x[1])
        self.chars, _ = zip(*count_pairs)
        self.vocab_size = len(self.chars)
        self.vocab = dict(zip(self.chars, range(len(self.chars))))
        with open(vocab_file, 'wb') as f:
            cPickle.dump(self.chars, f)
        self.tensor = np.array(list(map(self.vocab.get, data)))
        np.save(tensor_file, self.tensor)

    def load_preprocessed(self, vocab_file, tensor_file):
        with open(vocab_file, 'rb') as f:
            self.chars = cPickle.load(f)
        self.vocab_size = len(self.chars)
        self.vocab = dict(zip(self.chars, range(len(self.chars))))
        self.tensor = np.load(tensor_file)
        self.num_batches = int(self.tensor.size / (self.batch_size *
                                                   self.seq_length))

    def create_batches(self):
        self.num_batches = int(self.tensor.size / (self.batch_size *
                                                   self.seq_length))
        self.tensor = self.tensor[:self.num_batches * self.batch_size * self.seq_length]
        xdata = self.tensor
        ydata = np.copy(self.tensor)
        ydata[:-1] = xdata[1:]
        ydata[-1] = xdata[0]
        self.x_batches = np.split(xdata.reshape(self.batch_size, -1), self.num_batches, 1)
        self.y_batches = np.split(ydata.reshape(self.batch_size, -1), self.num_batches, 1)


    def next_batch(self):
        x, y = self.x_batches[self.pointer], self.y_batches[self.pointer]
        self.pointer += 1
        return x, y

    def reset_batch_pointer(self):
        self.pointer = 0

In [13]:
# -*- coding: utf-8 -*-
# Char-RNN 예제
# Author : solaris33
# Project URL : http://solarisailab.com/archives/2487
# GitHub Repository : https://github.com/solaris33/char-rnn-tensorflow/
# Reference : https://github.com/sherjilozair/char-rnn-tensorflow

import tensorflow as tf
import numpy as np

# 학습에 필요한 설정값들을 지정합니다.
data_dir = 'Documents/My/data/tinyshakespeare'
#data_dir = 'data/linux'
batch_size = 50 # Training : 50, Sampling : 1
seq_length = 50 # Training : 50, Sampling : 1
hidden_size = 128   # 히든 레이어의 노드 개수
learning_rate = 0.002
num_epochs = 2
num_hidden_layers = 2
grad_clip = 5   # Gradient Clipping에 사용할 임계값

# TextLoader를 이용해서 데이터를 불러옵니다.
data_loader = TextLoader(data_dir, batch_size, seq_length)
# 학습데이터에 포함된 모든 단어들을 나타내는 변수인 chars와 chars에 id를 부여해 dict 형태로 만든 vocab을 선언합니다.
chars = data_loader.chars 
vocab = data_loader.vocab
vocab_size = data_loader.vocab_size # 전체 단어개수

# 인풋데이터와 타겟데이터, 배치 사이즈를 입력받기 위한 플레이스홀더를 설정합니다.
input_data = tf.placeholder(tf.int32, shape=[None, None])  # input_data : [batch_size, seq_length])
target_data = tf.placeholder(tf.int32, shape=[None, None]) # target_data : [batch_size, seq_length])
state_batch_size = tf.placeholder(tf.int32, shape=[])      # Training : 50, Sampling : 1

# RNN의 마지막 히든레이어의 출력을 소프트맥스 출력값으로 변환해주기 위한 변수들을 선언합니다.
# hidden_size -> vocab_size
softmax_w = tf.Variable(tf.random_normal(shape=[hidden_size, vocab_size]), dtype=tf.float32)
softmax_b = tf.Variable(tf.random_normal(shape=[vocab_size]), dtype=tf.float32)

# num_hidden_layers만큼 LSTM cell(히든레이어)를 선언합니다.
cells = []
for _ in range(0, num_hidden_layers):
    cell = tf.nn.rnn_cell.BasicLSTMCell(hidden_size)
    cells.append(cell)

# cell을 종합해서 RNN을 정의합니다.
cell = tf.contrib.rnn.MultiRNNCell(cells, state_is_tuple=True)

# 인풋데이터를 변환하기 위한 Embedding Matrix를 선언합니다.
# hidden_size -> vocab_size
embedding = tf.Variable(tf.random_normal(shape=[vocab_size, hidden_size]), dtype=tf.float32)
inputs = tf.nn.embedding_lookup(embedding, input_data)

# 초기 state 값을 0으로 초기화합니다.
initial_state = cell.zero_state(state_batch_size, tf.float32)

# 학습을 위한 tf.nn.dynamic_rnn을 선언합니다.
# outputs : [batch_size, seq_length, hidden_size]
outputs, final_state = tf.nn.dynamic_rnn(cell, inputs, initial_state=initial_state, dtype=tf.float32)
# ouputs을 [batch_size * seq_length, hidden_size]] 형태로 바꿉니다.
output = tf.reshape(outputs, [-1, hidden_size])

# 최종 출력값을 설정합니다.
# logits : [batch_size * seq_length, vocab_size]
logits = tf.matmul(output, softmax_w) + softmax_b
probs = tf.nn.softmax(logits)

# Cross Entropy 손실 함수를 정의합니다. 
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=target_data))

# 옵티마이저를 선언하고 옵티마이저에 Gradient Clipping을 적용합니다.
# grad_clip(=5)보다 큰 Gradient를 5로 Clippin합니다.
tvars = tf.trainable_variables()
grads, _ = tf.clip_by_global_norm(tf.gradients(loss, tvars), grad_clip)
optimizer = tf.train.AdamOptimizer(learning_rate)
train_step = optimizer.apply_gradients(zip(grads, tvars))

# 세션을 열고 학습을 진행합니다.
with tf.Session() as sess:
    # 변수들에 초기값을 할당합니다.
    sess.run(tf.global_variables_initializer())
    
    for e in range(num_epochs):
        data_loader.reset_batch_pointer()
        # 초기 상태값을 지정합니다.
        state = sess.run(initial_state, feed_dict={state_batch_size : batch_size})

        for b in range(data_loader.num_batches):
            # x, y 데이터를 불러옵니다.
            x, y = data_loader.next_batch()
            # y에 one_hot 인코딩을 적용합니다. 
            y = tf.one_hot(y, vocab_size)            # y : [batch_size, seq_length, vocab_size]
            y = tf.reshape(y, [-1, vocab_size])       # y : [batch_size * seq_length, vocab_size]
            y = y.eval()

            # feed-dict에 사용할 값들과 LSTM 초기 cell state(feed_dict[c])값과 hidden layer 출력값(feed_dict[h])을 지정합니다.
            feed_dict = {input_data : x, target_data: y, state_batch_size : batch_size}
            for i, (c, h) in enumerate(initial_state):
                feed_dict[c] = state[i].c
                feed_dict[h] = state[i].h

            # 한스텝 학습을 진행합니다.
            _, loss_print, state = sess.run([train_step, loss, final_state], feed_dict=feed_dict)

            print("{}(학습한 배치개수)/{}(학습할 배치개수), 반복(epoch): {}, 손실함수(loss): {:.3f}".format(
                          e * data_loader.num_batches + b,
                          num_epochs * data_loader.num_batches,
                          (e+1), 
                          loss_print))

    print("트레이닝이 끝났습니다!")   
    

    # 샘플링 시작
    print("샘플링을 시작합니다!")
    num_sampling = 4000  # 생성할 글자(Character)의 개수를 지정합니다. 
    prime = u' '         # 시작 글자를 ' '(공백)으로 지정합니다.
    sampling_type = 1    # 샘플링 타입을 설정합니다.
    state = sess.run(cell.zero_state(1, tf.float32)) # RNN의 최초 state값을 0으로 초기화합니다.

    # Random Sampling을 위한 weighted_pick 함수를 정의합니다.
    def weighted_pick(weights):
        t = np.cumsum(weights)
        s = np.sum(weights)
        return(int(np.searchsorted(t, np.random.rand(1)*s)))

    ret = prime       # 샘플링 결과를 리턴받을 ret 변수에 첫번째 글자를 할당합니다.
    char = prime      # Char-RNN의 첫번쨰 인풋을 지정합니다.  
    for n in range(num_sampling):
        x = np.zeros((1, 1))
        x[0, 0] = vocab[char]

        # RNN을 한스텝 실행하고 Softmax 행렬을 리턴으로 받습니다.
        feed_dict = {input_data: x, state_batch_size : 1, initial_state: state}
        [probs_result, state] = sess.run([probs, final_state], feed_dict=feed_dict)         

        # 불필요한 차원을 제거합니다.
        # probs_result : (1,65) -> p : (65)
        p = np.squeeze(probs_result)

        # 샘플링 타입에 따라 3가지 종류로 샘플링 합니다.
        # sampling_type : 0 -> 다음 글자를 예측할때 항상 argmax를 사용
        # sampling_type : 1(defualt) -> 다음 글자를 예측할때 항상 random sampling을 사용
        # sampling_type : 2 -> 다음 글자를 예측할때 이전 글자가 ' '(공백)이면 random sampling, 그렇지 않을 경우 argmax를 사용
        if sampling_type == 0:
            sample = np.argmax(p)
        elif sampling_type == 2:
            if char == ' ':
                sample = weighted_pick(p)
            else:
                sample = np.argmax(p)
        else:
            sample = weighted_pick(p)

        pred = chars[sample]
        ret += pred     # 샘플링 결과에 현재 스텝에서 예측한 글자를 추가합니다. (예를들어 pred=L일 경우, ret = HEL -> HELL)
        char = pred     # 예측한 글자를 다음 RNN의 인풋으로 사용합니다.

    print("샘플링 결과:")
    print(ret)

reading text file
Instructions for updating:
This class is deprecated, please use tf.nn.rnn_cell.LSTMCell, which supports all the feature this cell currently has. Please replace the existing code with tf.nn.rnn_cell.LSTMCell(name='basic_lstm_cell').
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

0(학습한 배치개수)/892(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 5.116
1(학습한 배치개수)/892(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 4.171
2(학습한 배치개수)/892(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 3.896
3(학습한 배치개수)/892(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 3.947
4(학습한 배치개수)/892(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 3.649
5(학습한 배치개수)/892(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 3.565
6(학습한 배치개수)/892(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 3.578
7(학습한 배치개수)/892(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 3.448
8(학습한 배치개수)/892(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 3.304
9(학습한 배치개수)/892(학습할 배치개수), 반

123(학습한 배치개수)/892(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.984
124(학습한 배치개수)/892(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 2.017
125(학습한 배치개수)/892(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 2.062
126(학습한 배치개수)/892(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.979
127(학습한 배치개수)/892(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 2.011
128(학습한 배치개수)/892(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 2.058
129(학습한 배치개수)/892(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.958
130(학습한 배치개수)/892(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.940
131(학습한 배치개수)/892(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.999
132(학습한 배치개수)/892(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 2.032
133(학습한 배치개수)/892(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.971
134(학습한 배치개수)/892(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.958
135(학습한 배치개수)/892(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 2.016
136(학습한 배치개수)/892(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 2.006
137(학습한 배치개수)/892(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 2.024
138(학습한 배치개수)/892(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 2.026
139(학습한 배치개수)/892(학습할 배치

259(학습한 배치개수)/892(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.770
260(학습한 배치개수)/892(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.747
261(학습한 배치개수)/892(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.750
262(학습한 배치개수)/892(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.756
263(학습한 배치개수)/892(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.776
264(학습한 배치개수)/892(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.736
265(학습한 배치개수)/892(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.829
266(학습한 배치개수)/892(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.858
267(학습한 배치개수)/892(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.794
268(학습한 배치개수)/892(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.785
269(학습한 배치개수)/892(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.774
270(학습한 배치개수)/892(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.783
271(학습한 배치개수)/892(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.831
272(학습한 배치개수)/892(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.754
273(학습한 배치개수)/892(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.814
274(학습한 배치개수)/892(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.786
275(학습한 배치개수)/892(학습할 배치

395(학습한 배치개수)/892(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.703
396(학습한 배치개수)/892(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.641
397(학습한 배치개수)/892(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.625
398(학습한 배치개수)/892(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.680
399(학습한 배치개수)/892(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.656
400(학습한 배치개수)/892(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.718
401(학습한 배치개수)/892(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.699
402(학습한 배치개수)/892(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.690
403(학습한 배치개수)/892(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.729
404(학습한 배치개수)/892(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.700
405(학습한 배치개수)/892(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.705
406(학습한 배치개수)/892(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.730
407(학습한 배치개수)/892(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.663
408(학습한 배치개수)/892(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.741
409(학습한 배치개수)/892(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.671
410(학습한 배치개수)/892(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.715
411(학습한 배치개수)/892(학습할 배치

531(학습한 배치개수)/892(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.641
532(학습한 배치개수)/892(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.584
533(학습한 배치개수)/892(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.563
534(학습한 배치개수)/892(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.672
535(학습한 배치개수)/892(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.582
536(학습한 배치개수)/892(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.573
537(학습한 배치개수)/892(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.580
538(학습한 배치개수)/892(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.626
539(학습한 배치개수)/892(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.590
540(학습한 배치개수)/892(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.606
541(학습한 배치개수)/892(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.659
542(학습한 배치개수)/892(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.617
543(학습한 배치개수)/892(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.642
544(학습한 배치개수)/892(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.541
545(학습한 배치개수)/892(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.653
546(학습한 배치개수)/892(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.552
547(학습한 배치개수)/892(학습할 배치

666(학습한 배치개수)/892(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.570
667(학습한 배치개수)/892(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.575
668(학습한 배치개수)/892(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.599
669(학습한 배치개수)/892(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.572
670(학습한 배치개수)/892(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.532
671(학습한 배치개수)/892(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.530
672(학습한 배치개수)/892(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.622
673(학습한 배치개수)/892(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.565
674(학습한 배치개수)/892(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.589
675(학습한 배치개수)/892(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.615
676(학습한 배치개수)/892(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.615
677(학습한 배치개수)/892(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.575
678(학습한 배치개수)/892(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.579
679(학습한 배치개수)/892(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.604
680(학습한 배치개수)/892(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.601
681(학습한 배치개수)/892(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.579
682(학습한 배치개수)/892(학습할 배치

801(학습한 배치개수)/892(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.518
802(학습한 배치개수)/892(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.575
803(학습한 배치개수)/892(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.589
804(학습한 배치개수)/892(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.494
805(학습한 배치개수)/892(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.520
806(학습한 배치개수)/892(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.564
807(학습한 배치개수)/892(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.506
808(학습한 배치개수)/892(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.488
809(학습한 배치개수)/892(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.557
810(학습한 배치개수)/892(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.536
811(학습한 배치개수)/892(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.499
812(학습한 배치개수)/892(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.502
813(학습한 배치개수)/892(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.488
814(학습한 배치개수)/892(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.520
815(학습한 배치개수)/892(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.495
816(학습한 배치개수)/892(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.609
817(학습한 배치개수)/892(학습할 배치

In [15]:
tf.reset_default_graph()
data_dir = 'Documents/My/data/linux'
batch_size = 50 # Training : 50, Sampling : 1
seq_length = 50 # Training : 50, Sampling : 1
hidden_size = 128   # 히든 레이어의 노드 개수
learning_rate = 0.002
num_epochs = 2
num_hidden_layers = 2
grad_clip = 5   # Gradient Clipping에 사용할 임계값

# TextLoader를 이용해서 데이터를 불러옵니다.
data_loader = TextLoader(data_dir, batch_size, seq_length)
# 학습데이터에 포함된 모든 단어들을 나타내는 변수인 chars와 chars에 id를 부여해 dict 형태로 만든 vocab을 선언합니다.
chars = data_loader.chars 
vocab = data_loader.vocab
vocab_size = data_loader.vocab_size # 전체 단어개수

# 인풋데이터와 타겟데이터, 배치 사이즈를 입력받기 위한 플레이스홀더를 설정합니다.
input_data = tf.placeholder(tf.int32, shape=[None, None])  # input_data : [batch_size, seq_length])
target_data = tf.placeholder(tf.int32, shape=[None, None]) # target_data : [batch_size, seq_length])
state_batch_size = tf.placeholder(tf.int32, shape=[])      # Training : 50, Sampling : 1

# RNN의 마지막 히든레이어의 출력을 소프트맥스 출력값으로 변환해주기 위한 변수들을 선언합니다.
# hidden_size -> vocab_size
softmax_w = tf.Variable(tf.random_normal(shape=[hidden_size, vocab_size]), dtype=tf.float32)
softmax_b = tf.Variable(tf.random_normal(shape=[vocab_size]), dtype=tf.float32)

# num_hidden_layers만큼 LSTM cell(히든레이어)를 선언합니다.
cells = []
for _ in range(0, num_hidden_layers):
    cell = tf.nn.rnn_cell.BasicLSTMCell(hidden_size)
    cells.append(cell)

# cell을 종합해서 RNN을 정의합니다.
cell = tf.contrib.rnn.MultiRNNCell(cells, state_is_tuple=True)

# 인풋데이터를 변환하기 위한 Embedding Matrix를 선언합니다.
# hidden_size -> vocab_size
embedding = tf.Variable(tf.random_normal(shape=[vocab_size, hidden_size]), dtype=tf.float32)
inputs = tf.nn.embedding_lookup(embedding, input_data)

# 초기 state 값을 0으로 초기화합니다.
initial_state = cell.zero_state(state_batch_size, tf.float32)

# 학습을 위한 tf.nn.dynamic_rnn을 선언합니다.
# outputs : [batch_size, seq_length, hidden_size]
outputs, final_state = tf.nn.dynamic_rnn(cell, inputs, initial_state=initial_state, dtype=tf.float32)
# ouputs을 [batch_size * seq_length, hidden_size]] 형태로 바꿉니다.
output = tf.reshape(outputs, [-1, hidden_size])

# 최종 출력값을 설정합니다.
# logits : [batch_size * seq_length, vocab_size]
logits = tf.matmul(output, softmax_w) + softmax_b
probs = tf.nn.softmax(logits)

# Cross Entropy 손실 함수를 정의합니다. 
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=target_data))

# 옵티마이저를 선언하고 옵티마이저에 Gradient Clipping을 적용합니다.
# grad_clip(=5)보다 큰 Gradient를 5로 Clippin합니다.
tvars = tf.trainable_variables()
grads, _ = tf.clip_by_global_norm(tf.gradients(loss, tvars), grad_clip)
optimizer = tf.train.AdamOptimizer(learning_rate)
train_step = optimizer.apply_gradients(zip(grads, tvars))

# 세션을 열고 학습을 진행합니다.
with tf.Session() as sess:
    # 변수들에 초기값을 할당합니다.
    sess.run(tf.global_variables_initializer())
    
    for e in range(num_epochs):
        data_loader.reset_batch_pointer()
        # 초기 상태값을 지정합니다.
        state = sess.run(initial_state, feed_dict={state_batch_size : batch_size})

        for b in range(data_loader.num_batches):
            # x, y 데이터를 불러옵니다.
            x, y = data_loader.next_batch()
            # y에 one_hot 인코딩을 적용합니다. 
            y = tf.one_hot(y, vocab_size)            # y : [batch_size, seq_length, vocab_size]
            y = tf.reshape(y, [-1, vocab_size])       # y : [batch_size * seq_length, vocab_size]
            y = y.eval()

            # feed-dict에 사용할 값들과 LSTM 초기 cell state(feed_dict[c])값과 hidden layer 출력값(feed_dict[h])을 지정합니다.
            feed_dict = {input_data : x, target_data: y, state_batch_size : batch_size}
            for i, (c, h) in enumerate(initial_state):
                feed_dict[c] = state[i].c
                feed_dict[h] = state[i].h

            # 한스텝 학습을 진행합니다.
            _, loss_print, state = sess.run([train_step, loss, final_state], feed_dict=feed_dict)

            print("{}(학습한 배치개수)/{}(학습할 배치개수), 반복(epoch): {}, 손실함수(loss): {:.3f}".format(
                          e * data_loader.num_batches + b,
                          num_epochs * data_loader.num_batches,
                          (e+1), 
                          loss_print))

    print("트레이닝이 끝났습니다!")   
    

    # 샘플링 시작
    print("샘플링을 시작합니다!")
    num_sampling = 4000  # 생성할 글자(Character)의 개수를 지정합니다. 
    prime = u' '         # 시작 글자를 ' '(공백)으로 지정합니다.
    sampling_type = 1    # 샘플링 타입을 설정합니다.
    state = sess.run(cell.zero_state(1, tf.float32)) # RNN의 최초 state값을 0으로 초기화합니다.

    # Random Sampling을 위한 weighted_pick 함수를 정의합니다.
    def weighted_pick(weights):
        t = np.cumsum(weights)
        s = np.sum(weights)
        return(int(np.searchsorted(t, np.random.rand(1)*s)))

    ret = prime       # 샘플링 결과를 리턴받을 ret 변수에 첫번째 글자를 할당합니다.
    char = prime      # Char-RNN의 첫번쨰 인풋을 지정합니다.  
    for n in range(num_sampling):
        x = np.zeros((1, 1))
        x[0, 0] = vocab[char]

        # RNN을 한스텝 실행하고 Softmax 행렬을 리턴으로 받습니다.
        feed_dict = {input_data: x, state_batch_size : 1, initial_state: state}
        [probs_result, state] = sess.run([probs, final_state], feed_dict=feed_dict)         

        # 불필요한 차원을 제거합니다.
        # probs_result : (1,65) -> p : (65)
        p = np.squeeze(probs_result)

        # 샘플링 타입에 따라 3가지 종류로 샘플링 합니다.
        # sampling_type : 0 -> 다음 글자를 예측할때 항상 argmax를 사용
        # sampling_type : 1(defualt) -> 다음 글자를 예측할때 항상 random sampling을 사용
        # sampling_type : 2 -> 다음 글자를 예측할때 이전 글자가 ' '(공백)이면 random sampling, 그렇지 않을 경우 argmax를 사용
        if sampling_type == 0:
            sample = np.argmax(p)
        elif sampling_type == 2:
            if char == ' ':
                sample = weighted_pick(p)
            else:
                sample = np.argmax(p)
        else:
            sample = weighted_pick(p)

        pred = chars[sample]
        ret += pred     # 샘플링 결과에 현재 스텝에서 예측한 글자를 추가합니다. (예를들어 pred=L일 경우, ret = HEL -> HELL)
        char = pred     # 예측한 글자를 다음 RNN의 인풋으로 사용합니다.

    print("샘플링 결과:")
    print(ret)

loading preprocessed files
0(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 4.925
1(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 4.398
2(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 4.073
3(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 3.875
4(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 3.784
5(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 3.733
6(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 3.523
7(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 3.484
8(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 3.471
9(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 3.430
10(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 3.401
11(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 3.425
12(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 3.404
13(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 3.424
14(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 3.341
15(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 3.260
16(학습한 

135(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.905
136(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.934
137(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.939
138(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 2.001
139(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.987
140(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.975
141(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.998
142(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.915
143(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.933
144(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.950
145(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.944
146(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.917
147(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.939
148(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.983
149(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.880
150(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.979
151(학습한 

269(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.758
270(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.668
271(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.687
272(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.678
273(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.597
274(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.808
275(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.617
276(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.735
277(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.673
278(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.625
279(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.722
280(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.721
281(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.682
282(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.666
283(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.693
284(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.645
285(학습한 

403(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.581
404(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.623
405(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.526
406(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.786
407(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.766
408(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.722
409(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.707
410(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.636
411(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.684
412(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.527
413(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.449
414(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.479
415(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.440
416(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.470
417(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.488
418(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.529
419(학습한 

537(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.559
538(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.476
539(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.430
540(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.521
541(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.468
542(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.408
543(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.385
544(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.359
545(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.377
546(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.420
547(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.434
548(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.443
549(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.491
550(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.443
551(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.426
552(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.454
553(학습한 

670(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.331
671(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.349
672(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.474
673(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.397
674(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.383
675(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.370
676(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.340
677(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.359
678(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.342
679(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.324
680(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.359
681(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.346
682(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.358
683(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.360
684(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.350
685(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.336
686(학습한 

803(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.417
804(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.399
805(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.307
806(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.491
807(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.361
808(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.281
809(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.376
810(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.349
811(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.430
812(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.395
813(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.334
814(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.332
815(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.323
816(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.409
817(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.328
818(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.332
819(학습한 

936(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.295
937(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.272
938(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.314
939(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.352
940(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.392
941(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.402
942(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.375
943(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.287
944(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.320
945(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.350
946(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.352
947(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.286
948(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.251
949(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.342
950(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.350
951(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.390
952(학습한 